<h1 align='center'  style = 'margin-bottom: 0px'> The Pencil _Shaft_ Factory </h1>
<h2 align='center'  style = 'margin-top: 0px'> The GNU Linear Programming Kit (GLPK) </h2>
<p>&nbsp;</p>

We begin by __import__ing some Python modules we will use below:  Place your cursor in the cell below (i.e., select the cell), and then execute __[ctrl]-[enter]__.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np 

Also, you should have loaded a file called __1-MainExampleTools.py__ into the same folder as this notebook.  To see if that is the case, you can execute the next line which lists the files that are in the same folder as this notebook. 

In [ ]:
ls

If __1-MainExampleTools.py__ is there, then the next line should load the following commands: 

    add_method(object,function) -> adds function as a method of object)

    plotfill(ax, \*args, \*\*args) -> plot a curve and the region either above or below it

    xyaxes(ax,xlabel,ylabel) -> centered axes (best if last in sequence of plot commands)

Execute the next line by selecting and then __[ctrl]-[enter]__. 

In [ ]:
%run -i 1-MainExampleTools.py


## The Question 

<span style="font-size:larger"> 
Each day, Acme Mills receives one hardwood log which can be milled into as many as 20
thousand pencil shafts.  They sell standard pencil shafts to a pencil
factory for \$2 per thousand, and they sell engineering pencil shafts
to a drafting tools manufacturer for \$3 per thousand. It takes 1 hour to
make a thousand standard pencil shafts and two hours to make a thousand
engineering pencil shafts, and they have 4 milling machines each running for
7.5 hours each day. The pencil factory will buy all the standard shafts that
Acme makes up to 15 thousand each day. The drafting tools
manufacturer will buy all the engineering shafts Acme makes, but because not 
all their workers can make both types of shafts, the 
number of engineering shafts cannot exceed
twice the number of standard shafts. How many of each type of shaft should
they make in order to maximize their daily revenue?</span>

<p>&nbsp;</p>

## The Linear Program
Thus, we must solve the following problem: 
$$ maximize \; z = 2x+3y \quad subject \; to \; (x,y) \in \Omega $$ 

where the feasible region $\Omega$ is given by 
$$ \Omega = \left\lbrace (x,y)\in\mathbb{R}^2 \; \; \middle\vert
      \begin{array}{c} 
        x + \; y  \le  20 \\ 
        x + 2y  \le 30 \\
        x  \le 15 \\ 
        y  \le 2x \\ 
        x \ge 0, \quad y \ge 0
      \end{array}
    \right\rbrace 
$$
Similar to the way in which we stated the problem, there are 2 tasks we must address

1. Determining the structure of the Feasible Region. 

2. Finding where the largest value of $z$ occurs over the Feasible Region. 

We'll focus on the feasible region to begin with. 

<p>&nbsp;</p>

## The Feasible Region 
First, notice that all the constraints are _linear_, so that it is no surprise that the Feasible region both here and in general is a _polygon_ ( a _polyhedron_ in 3d and a _polytope_ in general).  

The sides (i.e., edges) of the polygon are not an issue -- we can and have easily drawn straight lines. 

In [ ]:
# Label the resulting feasible region
fig, axes = plt.subplots()
add_method(axes, plotfill)
add_method(axes, xyspines)

# redefine x
x  = np.linspace(0,15,20)

#plot the region satisfying the constraints
axes.plotfill( x, dict( a=1, b=1, c=20), color = 'blue'  )
axes.plotfill( x, dict( m = 2, b = 0 ), color = 'green' )
axes.plotfill( x, dict( a=1, b=2, c=30), color= 'red' )

# Label the Feasible Region 
FeasibleLabel = """
$\Omega: x+y \leq 20 $, 
     $  x + 2y \leq 30, $ 
     $  0 \leq y \leq 2x, $  
     $  0 \leq x \leq 15, $ """
axes.text(5,3, FeasibleLabel, fontsize = 14 )

#Show obvious vertices
axes.plot( [15,15],[0,5], color='brown')
axes.scatter([0,15,15],[0,0,5], color='brown', zorder = 5)

# Label the resulting feasible region
FeasibleLabel = """
$\Omega: x+y \leq 20 $, 
     $  x + 2y \leq 30, $ 
     $  0 \leq y \leq 2x, $  
     $  0 \leq x \leq 15, $ """
axes.text(5,3, FeasibleLabel, fontsize = 14 )
axes.text(15,5,' (15,5)', fontsize = 14)
axes.text(15,0,' (15,0)', fontsize = 14)
axes.text( 0,0,' (0,0)', fontsize = 14, ha = 'left', va = 'bottom')

# Plot the xy-axes and set upper bound on y
axes.set_ylim(0,20)
axes.set_xlim(0)

The vertices ("corners") on the other hand, are points in the feasible region for which some of the constraints become equalities. We show above three obvious vertices -- (0,0), (15,0), and (15,5).  

Constraints that are equalities for a given vertex are called _active constraints_ for that vertex -- which implies the remaining constraints are strict inequalities and are said to be _passive_.   

<h3 align="center" style="color:blue"> IMPORTANT: If a point $(x,y)$ satisfies a constraint as an <br/>_equality_, 
                                                  then that constraint is _active_ for that point</h3>

For example, the corner point (15,5) corresponds to the two _active constraints_ 
$$ x = 15 \quad and \quad x+y=20$$
The other constraints are _passive_, which corresponds to 
$$ 15 + 2(5) < 30, \;  5 < 2(15), \;  5 > 0, \; and \; 15 > 0 $$
 

### There may be many vertices!
An obvious approach in 2 dimensions is thus to take the constraints 2 at a time, set them to equalities, and then solve the resulting linear system. In general, in _r_ dimensions, given that we have $m>r$ constraints, we would take _r_ constraints at a time, set them to equalities, and then solve.  

However, there is a __HUGE__ problem with this approach.<p></p> 

<div align='center'> <b style = 'font-size:larger'> <i style = "color:blue"> There can be and often are very large numbers of vertices, <br/> many of which are not corner points for the feasible region! </i></b></div>

In general, the number of ways of choosing $r$ out of a possible $m$ constraints is given mathematically by 
$$ { m \choose r} = \frac {m!} { r! (m-r)!} \approx \frac {m^r}{r!} \quad as \quad m\rightarrow\infty$$
which shows that ${m \choose r}$ increases rapidly as $m$ increases.  For example, we have 6 constraints, which taken 2 at a time means we attempt to solve  
$$ { 6 \choose 2 } = 15 \; systems \; of\; equations$$
But a quick inspection shows that the feasible region is a polygon with only 5 vertices. 

<p>&nbsp;</p>

## AMPL, MATHPROG, GLPK, and GPLSOL 
There are at this point 2 ideas which should be rather obvious.  

1. We need methods that only calculate corner points for actual vertices of the Feasible region 

2. We need to use the constraints as "Mathematical Objects" that can be taken "$r$ at a time".   

Conventional programming cannot accomplish this 2nd task, even with the most powerful computer algebra systems and linear algebra libraries.  A linear program involves inequalities, which define infinite regions, thus implying that _the only truly applicable programming language is one based on sets and similar mathematical abstractions_. 

Thus, the OR community created for itself __A M__athematical __P__rogramming __L__anguage (AMPL).  The AMPL language is very large, quite abstract, and only partially implemented.  

Consequently, when OR researchers created the __G__nu __L__inear __P__rogramming __K__it (GLPK) that implemented a subset of the AMPL language, they used the term __MathProg__ to refer to the part of AMPL that is used by GLPK. (GNU is a project launched at MIT in the 1980's to produce software that is free and runs on most operating systems --  i.e., one of the original open source projects).   

Actually, GLPK does more than just MathProg, and the python library __pyomo__ has become an important tool to go beyond what GLPK + MathProg can solve.  We'll look at pyomo briefly near the end of the course, but for now, let's start playing with GLPK and MathProg.  

First, you need to realize that GLPK must be downloaded and installed (already is if you are using __ _CoCalc_ __.  In general, this means visiting either https://www.gnu.org/software/glpk/ or  http://winglpk.sourceforge.net/ for windows.   

The program __glpsol__ is part of GLPK.  If GLPK is installed correctly, executing the following cell should produce the corner point (15,5) as printed output.

<span style="color:#880000">NOTE: You may have to specify the path to glpsol. </span>

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : x  = 15;
eqn2 : x + y = 20;

# solve
solve;

# display results
display x, y;

end;

In particular, __%%script__ is _magic_ that runs programs that are __NOT__ a part of Python.  Specifically, 

    %%script  command  
    rest_of_cell

asks the operating system to run __command__ with the __rest_of_cell__ acting as the script that the __command__ is applied to.  For our purposes, 

    %%script glpsol -m /dev/stdin
    MathProg_Code
    
runs the program __glpsol__ on the the _standard input_ (named __stdin__).  The __MathProg_Code__ is correspondingly the code that __%%script__ sends to _standard input_, and thus what __glpsol__ runs on.   

Let's do that again, this time in order to find the vertex corresponding to the active constraints 
$$ x + y = 20,  \qquad x + 2y = 30 $$
The solution will satisfy the remaining constraints as strict inequalities.  

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : x +   y = 20;
eqn2 : x + 2*y = 30;

# solve
solve;

# display results
display x, y;

end;

If glpsol ran correctly, then it should have returned values of __x.val__ and __y.val__ as (10,10), respectively.   

In particular, we see that in MathProg, a "value" is a property of a variable.  That is, __glpsol__ is told that __x__ is a variable via the command 

    var x;
    
and upon completion, the value of __x__ corresponding to the solution is stored as __x.val__.  

Notice also that the equations are _mathematical objects_.  The first equation is named __eqn1__ and the second is __eqn2__.  

Now let's find the vertex corresponding to the active constraints 
$$ y = 2x \quad and \quad  x + 2y = 20 $$
Once again, we declare variables, define the equations, solve, and then display the results. 

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : y = 2*x;
eqn2 : x + 2*y = 30;

# solve
solve;

# display results
display x, y;

end;

Thus, we have corner points of 
$$ (0,0), (15,0), (15,5), (10,10), (6,12) $$ 
This allows us to plot only the feasible region.  

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)
add_method(axes, xyspines)

#plot the Feasible Region
axes.plot( x, 20 - x, x, 30 - 2*x, x, (30 - x)/2, zorder = 5 )
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12) ]
X,Y = zip(*vertices)
axes.fill(X,Y, facecolor='cyan', edgecolor='black', zorder = 0)

#plot the vertices
axes.scatter(X,Y, color='brown', zorder = 10)
for vert in vertices:
    axes.text( vert[0], vert[1], " ({0},{1})".format(vert[0], vert[1]), fontsize = 14)

# Label the resulting feasible region
FeasibleLabel = """
$\Omega: x+y \leq 20 $, 
     $  x + 2y \leq 30, $ 
     $  0 \leq y \leq 2x, $  
     $  0 \leq x \leq 15, $ """
axes.text(5,3, FeasibleLabel, fontsize = 14 )

# Plot the xy-axes and set upper bound on y
#axes.xyspines('$x$', '$y$', fontsize = 14)
axes.set_ylim(0,20)
axes.set_xlabel("x = 1000's of Standard shafts\n")
axes.set_ylabel("y = 1000's of Engineering shafts")

Note that we relied on visualization of $\Omega$ to avoid looking for 15 possible vertices.  Moreover, we only used __glpsol__ to find the vertices. 

But of course, __glpsol__ is actually designed to solve the entire linear program. 

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x >= 0, <= 15 ;  # we can define properties for
var y >= 0 ;  # the variables as we define them

# declare the objective 
maximize z: 2*x + 3*y;       ## DON'T FORGET THOSE SEMICOLONS!!!!! 

# subject to the constraints
s.t.    constraint1:   x +   y <=  20;
        constraint2:   x + 2*y <=  30;
        constraint3:         y <= 2*x; 

# solve
solve;

# display results
display x, y, z;

end;

We don't need to know exactly how GLPK solves this problem, but we do need at least some idea as to how GLPK avoids trying to find all possible vertices as it searches for the correct one.  We'll look at that next. 

<p>&nbsp;</p>

## Additional Examples

Let's use __glpsol__ to determine the feasible region for some other linear programs. 

<b style = "font-size:larger">Example 2 :</b> Find the feasible region -- including vertices -- for the following system of inequalities: 
\begin{eqnarray*}
      x + y \le 25 \\
      2x + y \le 30 \\
      x + 3y \le 45 \\
      x \ge 0, y \ge 0
\end{eqnarray*} 
First, let's plot the inequalities to identify which vertices we need to find. To do so, we first solve for $y$ in each:
\begin{eqnarray*} 
y \le 25 - x \\
y \le 30 - 2x \\
y \le 15 - \frac{x}{3}
\end{eqnarray*}
Then we plot the resulting inequalities: 

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

# redefine x
x  = np.linspace(0,15,100)

#plot the region satisfying the constraints
axes.plotfill( x, dict(a=1,b=1,c=25), color = 'blue', hatched = True, label = "$x+y \leq 25$" )
axes.plotfill( x, dict(a=2,b=1,c=30), color = 'green', hatched = True, label = "$2x+y \leq 30$" )
axes.plotfill( x, dict(a=1,b=3,c=45), color = 'red', hatched = True, label = "$x+3y \leq 45$" )

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)
axes.legend();

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

# redefine x
x  = np.linspace(0,15,100)

#plot the region satisfying the constraints
axes.plotfill( x, dict(a=1,b=1,c=25), color = 'blue', label = "$x+y \leq 25$" )
axes.plotfill( x, dict(a=2,b=1,c=30), color = 'green', label = "$2x+y \leq 30$" )
axes.plotfill( x, dict(a=1,b=3,c=45), color = 'red', label = "$x+3y \leq 45$" )

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)
axes.legend();

Clearly, we have the vertices (0,0), (0,15), and the x-intercept of $2x+y=30$, which is (15,0).  There are two vertices which are obviously __not__ in the feasible region -- those intersecting the line $x+y=25$.  Indeed, the constraint $x+y\le 25$ will always be passive.  

The only remaining vertex in the feasible region is the intersection of 
$$ 2x+y = 30 \qquad and \qquad  x + 3y = 45 $$ 
which we we find using __glpsol__. 

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : 2*x +   y = 30;
eqn2 :   x + 3*y = 45;

# solve
solve;

# display results
display x, y;

end;

Now let's add these vertices and their labels to the figure. 

In [ ]:
vertices = np.array( [ (0,0), (0,15), (9,12), (15,0)] ).T
axes.scatter(*vertices, color = 'brown', zorder = 10)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex), zorder = 10)
fig

If you prefer, you can also use the vertices to plot _only_ the feasible region. 

In [ ]:
fig, axes = plt.subplots()

vertices = np.array( [ (0,0), (0,15), (9,12), (15,0)] ).T
axes.fill(*vertices, facecolor = 'cyan', edgecolor = 'black', zorder = 0 )
axes.scatter(*vertices, color = 'brown', zorder = 1)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex),zorder = 1)

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14,  zorder = 1 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)

Let's look at a couple of our examples from the previous module: 

<b style = "font-size:larger">Example 3 (Overtime Cost):</b> Amco widget makers has 2 hourly employees -- Alice and Bob.  Alice makes \$20 per hour while Bob makes \$16 per hour, but Alice produces 25 widgets per hour while Bob produces only 20 widgets per hour.  Alice has seniority and thus must be given at least 1.5 hours of overtime for every 1 hour of overtime for Bob.  A new order for 100 widgets has just been placed as a priority rush job, so it will have to be filled using overtime.  How many hours of overtime should be alloted to Alice and Bob, respectively, if the goal is to minimize overtime pay and each is paid time and a half for each hour of overtime? 

__Find the vertices of the Feasible Region:__ We saw that we needed to solve the linear program
\begin{eqnarray*} 
minimize \;     & z = 30x+24y \\
subject \; to\; & 25x + 20y  \ge 30 \\
                & x  \ge 1.5y \\ 
                & x \ge 0, y \ge 0
\end{eqnarray*}
Let's plot the inequalities to see which vertices we need to obtain. 

Notice that we have "great than" inequalities in this application, so we use the __region__ option in __plotfill__:

__region = 'above'__  fills the region above the curve 

__region = 'below'__  fills the region below the curve 

__region = 'to_zero'__ fills the region between the curve and the x-axis

__region = 'above_zero'__ fills the region above the x-axis and below the curve (default)

__region = float__  fills the region between the curve and the __float__ing point number

We also notice that $x \ge 1.5y$ is the same as $x - 1.5y \ge 0$.  However, notice that points _below_ the line $x - 1.5y = 0$ satisfy the inequality -- such as, for example, the point (10,0).  Thus, we use the __below__ option for this constraint. 

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

# redefine x
x  = np.linspace(0,5,100)

#plot the region satisfying the constraints
axes.plotfill( x, dict(a=1, b=-1.5, c=0), region = 'below', color = 'green', hatched = True, label = "$x-1.5y \geq 0$" )
axes.plotfill( x, dict(a=25,b=20,c=30), region = 'above', color = 'red', hatched = True, label = "$25x+20y \geq 30$" )

# Label the resulting feasible region
axes.text(3,1, '$\Omega$', fontsize = 14 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)
axes.legend()

The only vertices are the x-intercept of the line $ 25x + 20y  = 30$, which is (1.2,0), and the intersection of the lines 
$$ 25x + 20y = 30 \qquad and \qquad x = 1.5 y $$
This is simple enough by hand, since 
$$  25(1.5y) + 20y = 30 \quad \implies \quad 57.5y =30 \quad \implies \quad y = 0.52174 $$
to 5 places after the decimal.  Correspondingly, $x = 1.5(0.52174) = 0.78261$, so the other vertex is $(0.78261,0.52174)$. 

Let's verify this using __glpsol__. 

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : 25*x + 20* y = 30;
eqn2 :    x = 1.5*y ;

# solve
solve;

# display results
display x, y;

end;

Let's add these two vertices to the figure: 

In [ ]:
vertices = np.array( [ (1.2,0), (0.78261,0.52174) ] ).T
axes.scatter(*vertices, color = 'brown', zorder = 10)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex), zorder = 10)
fig

Let's do one more: 

<b style = "font-size:larger">Example 4  (Advertising Duration)</b> Acme wants to advertise its products and is given a table describing the number of people (in thousands) expected to view either internet or tv advertising per day, along with the suggested targets for each demographic.   

| Age Demographic | 18 - 36 years | 36 - 54 | over 54 |  cost |
|:---------------:|:-------------:|:-------:|:-------:| -----:|
|   Internet      |        5      |    3    |    1    | \$800 |
|      TV         |        2      |    3    |    5    | \$900 |
| --------------- | ------------- | ------- | ------- | ----- |
|   Targets       |       24      |   30    |    18   |       |

If Acme allots \$10,000 to spend on their advertising campaign and wants to at least reach their targets, what decisions for internet and TV advertising maximizes the combined duration of the advertising campaign? 

__Solution:__ The linear program for this problem is 
\begin{eqnarray*} 
maximize \;      & z = x + y   & &\\
subject \; to \; & 800x + 900y & \le & 10,000 \\
                 &   5x +   2y & \ge & 24 \\ 
                 &   3x +   3y & \ge & 30 \\ 
                 &    x +   5y & \ge & 18 \\ 
                 & x \ge 0, & &y \ge 0 
\end{eqnarray*}
The first constraint requires that $y$ coordinates be __below__ the curve, whereas the remaining 3 constraints require that the $y$ coordinates be __above__ the curve. 

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

# redefine x
x  = np.linspace(0,15,100)

#plot the region satisfying the constraints
axes.plotfill( x, dict(a=800,b=900,c=10000), region = 'below', color = 'cyan',  
               hatched = True, label = "$800x+900y \leq 10,000$" )
axes.plotfill( x, dict(a=5, b=2, c=24), region = 'above', color = 'blue', 
               hatched = True, label = '$5x+2y \geq 24$' )
axes.plotfill( x, dict(a=3, b=3, c=30), region = 'above', color = 'green', 
               hatched = True, label = '$3x+3y\geq30$' )
axes.plotfill( x, dict(a=1, b=5, c=18), region = 'above', color = 'red', 
               hatched = True, label = '$x+5y \geq 18$')

# Label the resulting feasible region
axes.text(8,3, '$\Omega$', fontsize = 14 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,12)
axes.legend()

There are now 4 vertices that bound the polygonal feasible region.  The x-intercepts, y-intercepts and intersection below the blue line are __not__ in the feasible region. 

Let's use __glpsol__ to find these vertices. 

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn1 : 800*x + 900*y = 10000;
eqn2 :   5*x +   2*y =    24;

# solve
solve;

# display results
display x, y;

end;

In [ ]:
%%script glpsol -m /dev/stdin
    
# declare problem variables
var x;
var y;

# list all equations
eqn2 : 5*x + 2*y = 24;
eqn3 : 3*x + 3*y = 30;

# solve
solve;

# display results
display x, y;

end;

In [ ]:
%%script glpsol -m /dev/stdin
    
# declare problem variables
var x;
var y;

# list all equations
eqn3 : 3*x + 3*y = 30;
eqn4 :   x + 5*y = 18;

# solve
solve;

# display results
display x, y;

end;

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x;
var y;

# list all equations
eqn4 :     x +   5*y =    18;
eqn1 : 800*x + 900*y = 10000;

# solve
solve;

# display results
display x, y;

end;

The result is the 4 vertices $(0.55172, 10.62069)$, $(1.33333,8.66667)$, $(8,2)$, and $(10.90322,1.41935)$. Let's add them to our figure: 

In [ ]:
vertices = np.array( [ (0.55172,10.62069), (1.33333,8.66667), (8,2), (10.90322,1.41935) ] ).T
axes.fill(*vertices, facecolor = 'cyan', edgecolor = 'black', zorder = 5 )
axes.scatter(*vertices, color = 'brown', zorder = 10)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex),zorder = 10)

fig

<p>&nbsp;</p>

## Exercises 
See website for solutions -- this is section 1C. 

You determined the linear programs for each of the following (or look at the solutions): Now find the feasible region for each, by which we mean plot the polygon and determine the coordinates of the vertices of that polygon. 

1. A bakery sells pies and cakes.  Each pie costs \$2 to produce, requires a half an hour of labor, and sells for \$3; while each cake costs \$3, requires an hour of labor, and sells for \$5. How many pies and cakes, respectively, should be produced in each 8 hour day if the bakery spends no more than \$30 per day on supplies?   

2. A farmer will enclose a level rectangular field on 3 sides with a fence that costs \$5 per linear foot to build and \$1 per linear foot to maintain each year, but on the fourth side the field will be bounded by a wall that costs \$25 per linear foot to construct but is maintenance free after that.  What dimensions of the field will maximize its perimeter if the farmer wants to spend no more than \$10,000 total to enclose the field and \$1000 per year to maintain?   

3. Outdoor Furniture, Inc, makes benches and picnic tables. A bench requires 60 pounds of wood and requires 1 hour of labor.  A picnic table requires 80 pounds of wood and requires 2 hours of labor.  Their goal is to maximize the total number of benches and picnic tables produced per employee given that each employee is given 2000 pounds of wood each week and works 40 hours per week.  How many of each should they _decide_ to require of each employee? 

4. A farmer has $20$ acres on which he will plant $x$ acres of feed corn (for animals) and $y$ acres of sweet corn (for humans). It will cost \$1,000 per acre to grow feed corn and \$2,000 per acre to raise sweet corn; and the farmer has \$30,000 on hand at the beginning of the growing season.  Market analysis indicates that the number of acres of sweet corn should be no greater than twice the number of acres of feed corn, but he also wants to be taken seriously as a sweet corn producer, so he desires to grow no more than 15 acres of feed corn.  In units of thousands of dollars per acre, he knows that feed corn sells for \$2 thousands of dollars per acre, while sweet corn sells for \$5 thousands of dollars per acre. How many acres should he plant in feed corn, and how many in sweet corn, in order to maximize the revenue from the corn?  

5. Quality Furniture makes tables and chairs. The table below has the time required for each table and chair for cutting, assembly, and finishing, with the bottom row the minutes available each day for each (480 minutes = 8 hours).  

|     Item        | Cutting | Assembly | Finishing | profit |
|:---------------:|:-------:|:--------:|:---------:| ------:|
|    Table        |    60   |    45    |    48     |   \$90 |
|    Chair        |    20   |    30    |    24     |   \$10 |
| --------------- | ------- | -------- | --------- | ------ |
|   Available     |   480   |   480    |    480    |        | 

<div style = "margin-left:25px">Also, Quality Furniture needs  to make at least 4 chairs for each table.  The profit from the sale of each is shown in the final column.  How many tables and chairs, respectively, should they decide to make each day in order to maximize profit? </div>